In [28]:
!pip install rank_bm25

In [80]:
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

In [21]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
loader = PyPDFLoader("./data/SystemAdminGuide.pdf")
document = loader.load()

In [152]:
loader1 = CSVLoader("./data/final_data.csv")
document1 = loader1.load()

In [6]:
document = document[13:]
for i in range(len(document)):
    junk_text = f"Ericsson Internal  \nSYSTEM ADMINISTRATION GUIDE   \n{i+14} (334) \nPrepared (Subject resp)  No. \nZNIAISD Aditi Singh  1543 -AOT 101 5177 Uen  \nApproved (Document resp)  Checked  Date  Rev Reference  \n  2020 -11-04 PX4"
    document[i].page_content = document[i].page_content.replace(junk_text, "")

In [185]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = splitter.split_documents(document1)

In [22]:
embeddings = OpenAIEmbeddings()

c:\Users\Admin\.conda\envs\interview-prep\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [111]:
PROMPT_TEMPLATE = ("""
Use the following pieces of information to answer the user's question.
If you dont know the answer just say you know, don't try to make up an answer.

Context:{context}

Only return the helpful answer below and nothing else
Helpful answer
{question}                   
"""
)

In [112]:
prompt_template = PromptTemplate(input_variables=["context", "question"], template=PROMPT_TEMPLATE)

In [205]:
vector_store = FAISS.from_documents(text_chunks, embeddings)
retriever = vector_store.as_retriever(k=6)

In [206]:
retriever.get_relevant_documents("How to configure support groups in ITSM?")

[Document(page_content='subject: To create Support Groups\ncontent: On the\xa0Standard Configuration\xa0tab of the Application Administration Console, select the correct company.\nClick the\xa0Create\xa0link next to Support Group.\nEnter or select information in the\xa0Support Organization\xa0field. Enter the name of the support organization to create it. After you add the first support group to the organization, you can select the organization when you add the next support group.\nEnter or select information in the\xa0Support Group Name\xa0field.\nEnter or select the support group role. The\xa0Support Group Role\xa0field describes the role that the support group has in the organization. For example, the support group named Internal Support might provide the Help Desk function, while another support group provides tier 2 support.\n\xa0\nYou choose from the following support group roles, which are levels in the support group hierarchy:\nHelp Desk\nTier 1\nTier 2\nTier', metadata={'sourc

In [155]:
llm = ChatOpenAI(temperature=0.3)

In [187]:
keyword_retriever = BM25Retriever.from_documents(text_chunks)

In [188]:
keyword_retriever.k = 3

In [189]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, keyword_retriever], weights=[0.9, 0.1])

In [190]:
retrieval_chain = ({"context": ensemble_retriever, "question": RunnablePassthrough()} |
                   prompt_template |
                   llm |
                   StrOutputParser())

In [191]:
response = retrieval_chain.invoke("How to configure support groups in ITSM?")

In [192]:
response

'To configure support groups in ITSM, you need to go to the Application Administration Console, select the correct company, click the Create link next to Support Group, enter the Support Organization, Support Group Name, and select the Support Group Role. Repeat these steps to add all the support groups that you require.'

In [198]:
response1 = retrieval_chain.invoke("Incident Relationship to Emergency CR configuration")
response1

'A new configuration parameter has been added to specify whether an incident is required for an emergency change. This configuration form is accessible to users with "Infrastructure Change Config" permission.'

In [171]:
ensemble_retriever.get_relevant_documents("Can we make worklog mandatory in CR?")

[Document(page_content='subject: Worklog mandatory configuration\ncontent: Worklog mandatory is configurable for Completed and Cancelled status for change module. To configure navigate to Application Administration Console>Custom Configuration>Change Management>Rules', metadata={'source': './data/final_data.csv', 'row': 42}),
 Document(page_content='Critical and High \uf0e0 Worklog type “Install Results-Details” will be mandatory for only Critical and High change ticket.\nWorklog mandatory for ticket cancel \uf0e0Worklog type “Cancellation Information” is mandatory while cancelling a ticket.', metadata={'source': './data/final_data.csv', 'row': 42}),
 Document(page_content='a record will be created in this form and counting will show in worklog that how much CI Uploaded successfully. As part of the MRD 1904 add new field which will give all the CI name that getting failed. A new table also has been added with Ci Load ID and CI Name.', metadata={'source': './data/final_data.csv', 'row':

In [201]:
ensemble_retriever.get_relevant_documents("How to make Requested Start Date and Requested End Date mandatory in CR creation?", score=True)

[Document(page_content='subject: Configuration for Requested Dates Mandatory during Change Create.\ncontent: As part of the current CM B2B design “Requested Start Date” and “Requested End Date” is mandatory to create the CR irrespective of any change class and customer system need to send these two date/time values while sending the Cerate request. As per the OFR requirement these two date/time fields should not be mandatory to create the CR. So following changes need to be done in the existing design to fulfill the requirement.\nIn the form “Eric:CustomerInterface” 5 checkboxes has been created as. Refer below screen shot.\nNormal\nEmergency\nExpedited\nNo Impact\nStandard\nFor any customer if Requested Start date and End date need to be set as “Mandatory” for a particular change Class then the respective check box need to be checked. By Default For CR Type Latent Requested Start Data and End Dates are mandatory.', metadata={'source': './data/final_data.csv', 'row': 48}),
 Document(pa

In [202]:
retriever.invoke("How to make Requested Start Date and Requested End Date mandatory in CR creation?", score=True)

[Document(page_content='subject: Configuration for Requested Dates Mandatory during Change Create.\ncontent: As part of the current CM B2B design “Requested Start Date” and “Requested End Date” is mandatory to create the CR irrespective of any change class and customer system need to send these two date/time values while sending the Cerate request. As per the OFR requirement these two date/time fields should not be mandatory to create the CR. So following changes need to be done in the existing design to fulfill the requirement.\nIn the form “Eric:CustomerInterface” 5 checkboxes has been created as. Refer below screen shot.\nNormal\nEmergency\nExpedited\nNo Impact\nStandard\nFor any customer if Requested Start date and End date need to be set as “Mandatory” for a particular change Class then the respective check box need to be checked. By Default For CR Type Latent Requested Start Data and End Dates are mandatory.', metadata={'source': './data/final_data.csv', 'row': 48}),
 Document(pa